In [ ]:
import requests
import time
import hashlib
import random
from itertools import cycle
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib.request


In [ ]:
sessionid = '39673737047%3Acf6Ydg9lghnde6%3A23'
h = {'Accept-Language':'en-US,en;q=0.5',"Cookie":"sessionid={}".format(sessionid)}
# h['x-csrftoken']="DkPugukmMfCunkG7pDS6dS0T6wnZQqBX"
# h['content-type']='application/x-www-form-urlencoded'

In [ ]:
class Custom_Requests:
    use_proxy = None
    proxy_handler = None
    def __init__(self,use_proxy_param=False,use_session=True):
        if use_proxy_param:
            proxy_handler = PROXYHANDLER()
            self.proxy_handler = proxy_handler
        if use_session:
            self.session = HTMLSession()
        else:
            self.session = requests
            
    def get(self,url,parser="html.parser",request_type='get',headers=None
                     ,payload=None,params=None,try_times=20):
        retry_times = 0
        response = None
        while retry_times<=try_times:
            try:
                if self.proxy_handler:
                    proxy_option = self.proxy_handler.make_proxy()
                else:
                    proxy_option = None
                if request_type=='get':
                    response = self.session.get(url,proxies=proxy_option,headers=headers,
                                                data=payload,params=params)
                else:
                    response = self.session.post(url,proxies=proxy_option,headers=headers,
                                                data=payload,params=params)
                if response.ok:
                    break
            except:
                #traceback.print_exc()
                retry_times = retry_times + 1
        if response:
            if parser == 'json':
                response = response.json()
            elif parser == 'content':
                response = response.content
            elif parser == 'html.parser':
                response = BeautifulSoup(response.content,parser)
            elif parser == 'xml':
                response = BeautifulSoup(response.content,parser)
            else:
                response = response.content
        return response

class PROXYHANDLER():
    proxy_cycle = None
    proxies_list = []
    proxy_used = 0
    IP = None
    
    def __init__(self,recall_proxy_interval = 25,flag=False):
        self.flag = flag
        self.recall_proxy_interval = recall_proxy_interval
        try:    
            self.IP = self.get_ip_API()
        except:
            self.IP = self.get_ip()
        self.add_ip_to_whitelist(self.IP)
        self._call_for_tt_proxies()


    def _check_to_call_tt_proxy(self):
        if self.proxy_used % self.recall_proxy_interval == 0:
            self._call_for_tt_proxies()


    def _call_for_tt_proxies(self):
        try_times = 10
        while try_times != 0:
            try:
                self.obtain_proxy_list_from_tt_proxy()
                self.shuffel_proxies()
                break
            except:
                time.sleep(3)
                try_times = try_times - 1


    def shuffel_proxies(self,flag=False):
        random.shuffle(self.proxies_list)
        self.proxy_cycle = cycle(self.proxies_list)


    def make_proxy(self):
        PROXY = next(self.proxy_cycle)
        self.proxy_used = self.proxy_used + 1
        self._check_to_call_tt_proxy()
        return PROXY

    def get_proxy_list_from_tt_proxy(self):
        proxies = []
        secret = '50piVRZ9ZedEy2EfqXX9u3'
        params = {
            "license": "PBA5EE66C8FE5DE84",
            "time": int(time.time()),
            "cnt": 1000,
        }
        params["sign"] = hashlib.md5((params["license"] + str(params["time"]) + secret).encode('utf-8')).hexdigest()
        response = requests.get(
                    url="https://api.ttproxy.com/v1/obtain",
                    params=params,
                    headers={
                        "Content-Type": "text/plain; charset=utf-8",
                    },
                    data="1"
                ).json()
        proxies = response["data"]["proxies"]
        return proxies


    def obtain_proxy_list_from_tt_proxy(self,start=0,end=-1):
        proxies_list = self.get_proxy_list_from_tt_proxy()
        final_proxies = []
        for PROXY in proxies_list:
            if self.flag:
                pr = {"http"  : "http://" + PROXY}
            else:
                pr = {"http"  : "http://" + PROXY,"https" : "http://" + PROXY}
            final_proxies.append(pr)
        self.proxies_list = final_proxies


    def add_ip_to_whitelist(self,ip=''):
        secret = '50piVRZ9ZedEy2EfqXX9u3'
        params = {
        "license": "PBA5EE66C8FE5DE84",
        "time": int(time.time()),
        "cnt": 1000,
        }
        params["sign"] = hashlib.md5((params["license"] + str(params["time"]) + secret).encode('utf-8')).hexdigest()
        params['ip']=ip
        response = requests.get(
                url='https://api.ttproxy.com/v1/whitelist/add',
                params=params,
                headers={
                    "Content-Type": "text/plain; charset=utf-8",
                },
                data="1"
            ).json()
        return response

    def find_my_ip_address(self):
        ## importing socket module
        import socket
        ## getting the hostname by socket.gethostname() method
        hostname = socket.gethostname()
        ## getting the IP address using socket.gethostbyname() method
        ip_address = socket.gethostbyname(hostname)
        ## printing the hostname and ip_address
        # print(f"Hostname: {hostname}")
        # print(f"IP Address: {ip_address}")
        return ip_address

    def get_ip(self):
        import socket
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            # doesn't even have to be reachable
            s.connect(('10.255.255.255', 1))
            IP = s.getsockname()[0]
        except Exception:
            IP = '127.0.0.1'
        finally:
            s.close()
        return IP

    def get_ip_API(self):
        api_url = 'https://api.ipify.org?format=json'
        return requests.get(api_url).json()['ip']

    def get_ip_address_by_google(self):
        from requests_html import HTMLSession
        from bs4 import BeautifulSoup
        sess = HTMLSession()
        rrr = "https://www.google.com/search?ei=uihSYKirGpPB3LUP_Li3gAk&q=what+is+my+ip"
        res = sess.get(rrr).content
        res = BeautifulSoup(res,"lxml")
        res = res.find("span",style="font-size:20px").text
        return res


def find(key, dictionary):
    if isinstance(dictionary, dict):
        for k, v in dictionary.items():
            if k == key:
                yield v
            elif isinstance(v, dict):
                for result in find(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in find(key, d):
                        yield result
    else:
        if isinstance(dictionary, list):
            for d in dictionary:
                for result in find(key, d):
                    yield result


def get_images_from_hashtag_page(hashtag="nightsky", scroll_page=3):
    hashtag_url = f"https://www.instagram.com/explore/tags/{hashtag}/?__a=1"
    next_page_url = ""
    complete_url = hashtag_url+next_page_url
    goto_next_page = True
    image_urls_list = []
    while goto_next_page and scroll_page > 0:
        complete_url = hashtag_url+next_page_url
        #response = requests.get(complete_url,headers=h).json()
        response = cr.get(complete_url, headers=h, parser='json')
#         response = requests.get(complete_url).json()
        page_info = response['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]
        if page_info["has_next_page"]:
            end_cursor = page_info["end_cursor"]
            complete_url = "&max_id={}".format(end_cursor)
        else:
            goto_next_page = False
        edges = response['graphql']['hashtag']["edge_hashtag_to_media"]["edges"]
        image_urls = [a["node"]["display_url"] for a in edges]
        image_urls_list = image_urls_list+image_urls
        scroll_page = scroll_page - 1
        time.sleep(random.randint(10,20))
    return list(set(image_urls_list))



In [ ]:
cr = Custom_Requests(use_proxy_param=True, use_session=True)
cr.proxy_handler.IP

In [ ]:
igs = get_images_from_hashtag_page("toonapp",100)

In [21]:
from datetime import date
today = date.today()
print("Today date is: ", today)


Today date is:  2022-03-22


In [ ]:
with open(f"data/sheets/data-{today}.txt", 'w') as f:
    for s in igs:
        f.write(str(s) + '\n')


In [22]:
today = '2022-03-22'

In [ ]:
with open(f"data/sheets/data-{today}.txt", 'r') as f:
    data = [line.rstrip('\n') for line in f]


In [ ]:

for img in data:       
    urllib.request.urlretrieve(
        img, "00000001.jpg")


In [ ]:
data[0]